<a href="https://colab.research.google.com/github/HuyenNguyenHelen/MyTREC_2021/blob/main/BERT_Innitial_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers rank_bm25

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.7/dist-packages (2.0.0)
Requirement already up-to-date: rank_bm25 in /usr/local/lib/python3.7/dist-packages (0.2.1)


# Documents and Query processing

### Documents

In [2]:
import pandas as pd
path = r"/content/Source_NER-BERN_Query_gene-disease-drug_2021.csv"
with open(path, 'r', encoding = 'utf-8') as f:
    data = pd.read_csv(f)

column_names = [name.split(':')[0].replace('\'', '') for name in data.columns]
column_names = [name.split(':')[0].replace(' ', '') for name in column_names]
data.columns = column_names
print(data.columns )

# Creating new data with some txt fields
new_data = data[['brief_title', 'official_title', 'brief_summary_text_block', 'concat_string']]
new_data['brief_summary_text_block'] = new_data['brief_summary_text_block'].apply(lambda x: x.split(':')[1].replace("'",''))
new_data['brief_summary_text_block'].head()

Index(['3\tnct01064011\t22.094044\t{download_date', 'url', 'link_text',
       'org_study_id', 'secondary_id', 'nct_id', 'brief_title',
       'official_title', 'source', 'brief_summary_text_block',
       'detailed_description_text_block', 'primary_outcome_measure',
       'primary_outcome_description', 'secondary_outcome_measure',
       'secondary_outcome_description', 'arm_group_description',
       'intervention_description', 'start_date', 'primary_completion_date',
       'study_type', 'primary_purpose', 'condition', 'intervention_type',
       'intervention_name', 'gender', 'minimum_age', 'maximum_age',
       'eligibility_text_block', 'last_name', 'affiliation', 'country',
       'verification_date', 'study_first_submitted',
       'study_first_submitted_qc', 'last_update_posted', 'keyword',
       'mesh_term', 'intervention_browse_0_mesh_term',
       'intervention_browse_1_mesh_term', 'concat_string'],
      dtype='object')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


0     evalu recombin tissu plasminogen activ rtpa u...
1     phase clinic trial safeti dose determin pneum...
2     annual almost 5000 extrem low birth weight ou...
3     followup open label singlecent phase clinic t...
4     overal object phase iii clinic trial obtain i...
Name: brief_summary_text_block, dtype: object

In [3]:
doc_sum = new_data['brief_summary_text_block'].values.tolist()

### Query

In [4]:
# Loading the Query [2016/2021]
## Query 2016
import pandas as pd
import xml.etree.ElementTree as ET

tree = ET.parse(r"/content/topics2016.xml")
root = tree.getroot()

queryID, note, description, summary=[],[],[], []

 
i=0
while i<len(root):
    queryID.append(i+1)
    for j in range (len(root[i])):
        if root[i][j].tag == 'note':
            note.append(root[i][j].text.replace('\n',''))
        if root[i][j].tag == 'summary':
            summary.append(root[i][j].text)
        if root[i][j].tag == 'description':
            description.append(root[i][j].text)
  
    i+=1
    
queries = pd.DataFrame(zip(queryID, note, description, summary), columns =['queryID', 'note', 'description', 'summary'] )
queries.head()

,queryID,note,description,summary
0,1,78 M w/ pmh of CABG in early [**Month (only...,78 M transferred to nursing home for rehab aft...,A 78 year old male presents with frequent stoo...
1,2,Ms [**Known patient lastname 241**] is a [*...,An elderly female with past medical history of...,An elderly female with past medical history of...
2,3,Pt is a 75F with a PMHx significant for sev...,"A 75F with a PMHx significant for severe PVD, ...",A 75F found to be hypoglycemic with hypotensio...
3,4,The patient is an 87 yo woman with h/o oste...,"An 87 yo woman with h/o osteoporosis, multiple...","An 87 yo woman with h/o osteoporosis, DM2, dem..."
4,5,"An 82 M with COPD, s/p bioprosthetic AVR fo...",An 82 man with chronic obstructive pulmonary d...,An 82 man with multiple chronic conditions and...


# Question-Answering using Simple Wikipedia Index

This examples demonstrates the setup for Query / Question-Answer-Retrieval.

You can input a query or a question. The script then uses semantic search
to find relevant passages in Simple English Wikipedia (as it is smaller and fits better in RAM).

For semantic search, we use SentenceTransformer('msmarco-distilbert-base-v2') and retrieve
100 potentially passages that answer the input query.

Next, we use a more powerful CrossEncoder (cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6')) that
scores the query and all retrieved passages for their relevancy. The cross-encoder is neccessary to filter out certain noise
that might be retrieved from the semantic search step.


In [5]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

# We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
model_name = 'msmarco-MiniLM-L-6-v3'
bi_encoder = SentenceTransformer(model_name)
top_k = 100  # Number of passages we want to retrieve with the bi-encoder

# The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
        
# Here, we compute the corpus_embeddings from scratch (which can take a while depending on the GPU)
corpus_embeddings = bi_encoder.encode(doc_sum, convert_to_tensor=True, show_progress_bar=True)
print(corpus_embeddings)


tensor([[-0.4338,  0.1051, -0.2585,  ..., -0.1953,  0.8328, -0.2510],
        [-0.1596,  0.4130,  0.4209,  ...,  0.0819,  0.1962, -0.0152],
        [-0.0275,  0.4642,  0.6757,  ...,  0.4019,  0.5105,  0.0949],
        ...,
        [-0.2096,  0.1624,  0.0721,  ..., -0.5604,  0.3457,  0.6507],
        [ 0.0894,  0.0940,  0.2348,  ..., -0.2166,  0.4410,  0.2848],
        [ 0.0531, -0.4342,  0.4027,  ...,  0.1789, -0.0911,  0.1341]],
       device='cuda:0')


In [10]:
# This function will search all wikipedia articles for passages that
# answer the query
def search(query):

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, doc_sum[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-3 Bi-Encoder Retrieval hits")
    hits_en = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits_en[0:3]:
         print("\t{:.3f}\t{}".format(hit['score'], doc_sum[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-3 Cross-Encoder Re-ranker hits")
    hits_re = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits_re[0:3]:
        print("\t{:.3f}\t{}".format(hit['cross-score'], doc_sum[hit['corpus_id']].replace("\n", " ")))
    #return hits_re['cross-score'], doc_sum[int(hits_re['corpus_id'])]






In [11]:
search(query = "A 78 year old male presents with frequent stools and melena")


-------------------------

Top-3 Bi-Encoder Retrieval hits
	0.358	 70 year age schedul joint replac surgeri random tramacet naloxon plu morphin pca morphin pca ass qualiti analgesia postop period primari object determin opioid use combin use oral tramacet naloxon infus periop elderli 70 yr older elect knee hip arthroplasti spinal anesthesia secondari object determin incid va score ≥4 advers effect nausea vomit sedat respiratori depress pruritu confus time independ mobil hypothesi periop naloxon infus tramacet provid adequ analgesia elderli undergo total kneehip arthroplasti associ 80 reduct opioid use reduc opioidinduc side effect
	0.352	 purpos examin variat physic physiolog characterist accord sport age adolesc adult male team handbal th player adolesc n19 age 181±14 yr adult n23 286±57 yr player examin anthropometr characterist somatotyp bodi composit perform physic work capac forceveloc later scapular slide lsst shark skill davi squat jump sj countermov vertic jump without cmj
	0.

In [17]:
top_relevance = queries['summary'].apply(lambda x:search(query = x ))
top_relevance


-------------------------

Top-3 Bi-Encoder Retrieval hits
	0.361	 70 year age schedul joint replac surgeri random tramacet naloxon plu morphin pca morphin pca ass qualiti analgesia postop period primari object determin opioid use combin use oral tramacet naloxon infus periop elderli 70 yr older elect knee hip arthroplasti spinal anesthesia secondari object determin incid va score ≥4 advers effect nausea vomit sedat respiratori depress pruritu confus time independ mobil hypothesi periop naloxon infus tramacet provid adequ analgesia elderli undergo total kneehip arthroplasti associ 80 reduct opioid use reduc opioidinduc side effect
	0.330	 purpos examin variat physic physiolog characterist accord sport age adolesc adult male team handbal th player adolesc n19 age 181±14 yr adult n23 286±57 yr player examin anthropometr characterist somatotyp bodi composit perform physic work capac forceveloc later scapular slide lsst shark skill davi squat jump sj countermov vertic jump without cmj
	0.

TypeError: ignored